<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Sin-Dientes/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install docker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 7.1 MB/s eta 0:00:00


In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=f7f58e39f79f85ac0a56686b7fc5d526692334578d53b03e8ff9bbb4fd6f05e9
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [10]:
import findspark

In [11]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [12]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [13]:
ls

Dataset/  spark-3.3.2-bin-hadoop3.tgz


In [14]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [15]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [16]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [17]:
!echo $JAVA_HOME

/usr/lib/jvm/java-11-openjdk-amd64


In [18]:
!pip install -q findspark

In [19]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [21]:
import findspark

In [22]:
findspark.init()

In [23]:
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [24]:
from pyspark.sql import SparkSession

In [26]:
from pyspark.sql.functions import col


In [56]:
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

In [57]:
spark

In [58]:
import pandas as pd

In [59]:
# Creando el dataframe

In [60]:
df_mails = spark.read.load('/content/Dataset/email.snappy.parquet',format='parquet',sep=',',inferschema='true',header='true')

In [61]:
df_address = spark.read.load('/content/Dataset/address.snappy.parquet',format='parquet',sep=',',inferschema='true',header='true')

In [62]:
df_customer = spark.read.load('/content/Dataset/customer.snappy.parquet',format='parquet',sep=',',inferschema='true',header='true')

In [63]:
df_customerb = spark.read.load('/content/Dataset/customer_basics.snappy.parquet',format='parquet',sep=',',inferschema='true',header='true')

In [64]:
df_phones = spark.read.load('/content/Dataset/phones.snappy.parquet',format='parquet',sep=',',inferschema='true',header='true')

In [65]:
df_phones.show()

+-----------+--------------+-------------------+-----------------+----------+----------------+---------------+-------------+-------------------+-----------------+-------------+------------+---------------+-----------------------+------------------+---------------------+-----------------+----------+-----------+--------------------------+---------------------+--------------------+---------------------+----------------+-------------------------+-------------------------+------------------+-------------------+-----------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+------------------------------+----------------------------+-------------------+----------------+----------------+-------------------+--------------------+-----------------------+---------------------+
|customer_id|phone_use_type|address_sequence_id|phone_sequence_id|phone_type|phone_country_id|prefix_phone_id|phone_area_id|cellphone_p

In [66]:
df_customer_mails = df_customerb.join(df_mails, ['customer_id'], how='left')

In [67]:
df_customer_mails.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- personal_type: string (nullable = true)
 |-- personal_id: string (nullable = true)
 |-- admission_date: date (nullable = true)
 |-- sale_channel_id: string (nullable = true)
 |-- operation_channel_id: string (nullable = true)
 |-- branch_id: integer (nullable = true)
 |-- business_area_id: integer (nullable = true)
 |-- campaing_id: decimal(2,0) (nullable = true)
 |-- customer_situation_type: integer (nullable = true)
 |-- customer_condition_type: integer (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- tracking_manager_id: string (nullable = true)
 |-- credit_manager_id: string (nullable = true)
 |-- credit_recovery_manager_id: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- second_last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- marital_status_type: integer (nullable = true)
 |-- gender_type: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |

In [68]:
df_customer_mails.show()

+-----------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+----------+----------------+-----------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+--------------------------+----